In [1]:
import numpy as np
import gym 
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import mean_squared_error
from matplotlib import pyplot as plt


In [6]:
# This class contains information and control flow for an agent to make decisions based off experiences using DQN

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.n_actions = action_size 
        # we define some parameters and hyperparameters:
        # "lr" : learning rate
        # "gamma": discounted factor
        # "exploration_proba_decay": decay of the exploration probability
        # "batch_size": size of experiences we sample to train the DNN
        self.lr = 0.001
        self.gamma = 0.99
        self.exploration_proba = 1.0
        self.exploration_proba_decay = 0.005 
        self.batch_size = 32 

        # We define our memory buffer where we will store our experiences
        # We stores only the 2000 last time steps
        self.memory_buffer= list()
        self.max_memory_buffer = 2000
        
        # We creaate our model having to hidden layers of 24 units (neurones)
        # The first layer has the same size as a state size
        # The last layer has the size of actions space
        self.model = Sequential([
            Dense(units=24,input_dim=state_size, activation = 'relu'),
            Dense(units=24,activation = 'relu'),
            Dense(units=action_size, activation = 'linear')
        ])
        self.model.compile(loss="mse",
                      optimizer = Adam(learning_rate=self.lr))
        
    # The agent computes the action to perform given a state 
    def compute_action(self, current_state):
        # epsilon greedy policy:
        if np.random.uniform(0,1) < self.exploration_proba:
            return np.random.choice(range(self.n_actions))
        q_values = self.model.predict(current_state)[0]
        return np.argmax(q_values)
    
    # when an episode is finished, we update the exploration probability using 
    # espilon greedy algorithm
    def update_exploration_probability(self):
        self.exploration_proba = self.exploration_proba * np.exp(-self.exploration_proba_decay)
        print(self.exploration_proba)

    # At each time step, we store the corresponding experience
    def store_episode(self,current_state, action, reward, next_state, done):
        #We use a dictionnary to store them
        self.memory_buffer.append({
            "current_state":current_state,
            "action":action,
            "reward":reward,
            "next_state":next_state,
            "done" :done
        })
        # If the size of memory buffer exceeds its maximum, we remove the oldest experience
        if len(self.memory_buffer) > self.max_memory_buffer:
            self.memory_buffer.pop(0)

    # At the end of each episode, we train our model
    def train(self):
        # We shuffle the memory buffer and select a batch size of experiences
        np.random.shuffle(self.memory_buffer)
        batch_sample = self.memory_buffer[0:self.batch_size]
        
        # We iterate over the selected experiences
        for experience in batch_sample:
            # We compute the Q-values of S_t
            q_current_state = self.model.predict(experience["current_state"])
            # We compute the Q-target using Bellman optimality equation
            q_target = experience["reward"]
            if not experience["done"]:
                q_target = q_target + self.gamma*np.max(self.model.predict(experience["next_state"])[0])
            q_current_state[0][experience["action"]] = q_target
            # train the model
            self.model.fit(experience["current_state"], q_current_state, verbose=0)

In [3]:
env = gym.make("CartPole-v1")

In [8]:
state_size = env.observation_space.shape[0]
action_size = env.action_space.n 
n_episodes = 400 
max_iteration_ep = 500 
agent = DQNAgent(state_size, action_size)
total_steps = 0
batch_size = 20

In [11]:
# Iterate over episodes:
for e in range(n_episodes):
    current_state = env.reset()
    current_state = np.array([current_state])
    for step in range(max_iteration_ep):
        total_steps += 1
        # choose which action to perform using the agent:
        action = agent.compute_action(current_state)
        # the envrionment runs the action and returns the next state, a reward and whether the agent is done
        next_state, reward, done, _ = env.step(action)
        next_state = np.array([next_state])

        # We store each experience in a memory buffer to be used by the Agent to refit the Q funciton:
        agent.store_episode(current_state, action, reward, next_state, done)

        # if the episode is ended, we leave the loop after updating the exploration probability
        if done:
            agent.update_exploration_probability()
            break
        current_state = next_state

    # if the have at least batch_size experiences in the memory buffer
    # than we tain our model
    if total_steps >= batch_size:
        agent.train()


0.9900498337491681
1/1 [==============================] - 0s 12ms/step


2023-03-13 11:24:39.085246: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 10ms/step
0.9851119396030628
1/1 [==============================] - 0s 10ms/step
0.9801986733067554
1/1 [==============================] - 0s 11ms/step
0.9753099120283327
1/1 [==============================] - 0s 10ms/step
0.9704455335485083
1/1 [==============================] - 0s 9ms/step
0.9656054162575666
1/1 [==============================] - 0s 10ms/step
0.9607894391523233
1/1 [==============================] - 0s 11ms/step
0.9559974818331
1/1 [==============================] - 0s 10ms/step
0.951229424500714
1/1 [==============================] - 0s 10ms/step
0.9464851479534839
1/1 [==============================] - 0s 11ms/step
0.9417645335842488
1/1 [==============================] - 0s 10ms/step
0.9370674633774035
1/1 [==============================] - 0s 11ms/step
0.9323938199059484
1/1 [==============================] - 0s 10ms/step
0.9277434863285531
1/1 [==============================] - 0s 10ms/step
0.923116346386636
1/1 [=======

In [12]:
def make_video():
    env_to_wrap = gym.make('CartPole-v1')
    env = wrappers.Monitor(env_to_wrap, 'videos', force = True)
    rewards = 0
    steps = 0
    done = False
    state = env.reset()
    state = np.array([state])
    while not done:
        action = agent.compute_action(state)
        state, reward, done, _ = env.step(action)
        state = np.array([state])            
        steps += 1
        rewards += reward
    print(rewards)
    env.close()
    env_to_wrap.close()
make_video()


NameError: name 'wrappers' is not defined